In [2]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

In [5]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "mps")

In [6]:
labels = [0,0,0,0,0,0,0,0,0,0,0,0,0] #need to define

# Define VGG model
vgg = models.vgg16(pretrained=True)
vgg.features[0] = nn.Conv2d(3, 64, kernel_size=3, padding=1)  # Modify the first layer to accept 3 channels
vgg.classifier[-1] = nn.Linear(in_features=4096, out_features=len(labels))  # Modify the last layer for multi-label classification

vgg.to(device)

# Define loss function
criterion = nn.BCELoss()

# Define optimizer
optimizer = optim.Adam(vgg.parameters())

# Print the model architecture
print(vgg)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/abigail/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:25<00:00, 21.7MB/s] 


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [12]:
from data_proccessing import train_loader, valid_loader,train_loader, labels

vgg.train()
vgg.to(device)
# Retrain
num_epochs = 3
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = vgg(images.to(device))
        loss = criterion(outputs, labels.to(device))
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Display
        if (i+1) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, \
                                                                      num_epochs, \
                                                                      i+1, \
                                                                      len(train_loader), \
                                                                      loss.item()))

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>


RuntimeError: Invalid buffer size: 8.00 GB

In [8]:
from data_proccessing import train_loader, valid_loader,train_loader, labels
from torchvision.models import densenet121
from tqdm import tqdm
import torch.nn as nn
import torch
import torch.optim as optim


def train_model(model, train_loader, valid_loader, num_epochs=10, learning_rate=0.001, output_size=10):
    
    # Define device
    device = torch.device("cuda" if torch.cuda.is_available() else "mps")
    # Modify the classifier to match the number of output classes
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Linear(num_ftrs, output_size)#
    
    # Send model to device
    model = model.to(device)
    
    # Define loss function and optimizer
    # criterion = nn.BCEWithLogitsLoss()
    criterion = nn.CrossEntropyLoss()  # Assuming it's a classification task
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs} - Training'):
            # images, labels = images.to(device), labels.to(device)
            # Zero the parameter gradients
            optimizer.zero_grad()
            # Forward pass
            
            outputs = model(images)
            
            # Calculate loss
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Update running loss
            running_loss += loss.item() * images.size(0)
        
        # Calculate average training loss
        train_loss = running_loss / len(train_loader.dataset)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in tqdm(valid_loader, desc=f'Epoch {epoch + 1}/{num_epochs} - Validation'):
                images, labels = images.to(device), labels.to(device)
                
                # Forward pass
                outputs = model(images)
                
                # Calculate loss
                loss = criterion(outputs, labels)
                
                # Update validation loss
                val_loss += loss.item() * images.size(0)
        
        # Calculate average validation loss
        val_loss = val_loss / len(valid_loader.dataset)
        
        # Print training and validation loss
        print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
    
    print('Finished Training')
    
